In [1]:
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import Transformer, RNN, RETAIN
from pyhealth.tasks import mortality_prediction_mimic3_fn, drug_recommendation_mimic3_fn
from pyhealth.trainer import Trainer

dataset = MIMIC3Dataset(
    root='/data/physionet.org/files/mimiciii/1.4/',
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    code_mapping={
        # "ICD9CM": "CCSCM", 
        # "ICD9PROC": "CCSPROC",
        "NDC": ("ATC", {"target_kwargs": {"level": 4}})
        },
    dev=False,
    refresh_cache=True
)


/home/pj20/miniconda3/envs/txgnn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mimic3_ds = dataset.set_task(mortality_prediction_mimic3_fn)

train_dataset, val_dataset, test_dataset = split_by_patient(
    mimic3_ds, [0.8, 0.1, 0.1]
)
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

Generating samples for mortality_prediction_mimic3_fn: 100%|██████████| 46520/46520 [00:00<00:00, 53705.41it/s] 


In [3]:
model_w_pre = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    pretrained_emb="KG",
    embedding_dim=256,
)

Loading pretrained embedding for conditions...


FileNotFoundError: [Errno 2] No such file or directory: '/home/pj20/.cache/pyhealth/medcode/embeddings/KG/special_tokens/special_tokens.json'

In [5]:
# STEP 4: define trainer
trainer = Trainer(model=model_w_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="roc_auc",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(4031, 1536, padding_idx=0)
    (procedures): Embedding(1276, 1536, padding_idx=0)
    (drugs): Embedding(472, 1536, padding_idx=0)
  )
  (linear_layers): ModuleDict(
    (conditions): Linear(in_features=1536, out_features=256, bias=True)
    (procedures): Linear(in_features=1536, out_features=256, bias=True)
    (drugs): Linear(in_features=1536, out_features=256, bias=True)
  )
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
    (drugs): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=384, out_features=1, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 32
Optimizer: <class 'torch.opt

Epoch 0 / 10: 100%|██████████| 245/245 [00:02<00:00, 104.30it/s]

--- Train epoch-0, step-245 ---
loss: 0.2499



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 316.70it/s]

--- Eval epoch-0, step-245 ---
pr_auc: 0.1056
roc_auc: 0.6051
f1: 0.0000
loss: 0.2285
New best roc_auc score (0.6051) at epoch-0, step-245


...

--- Train epoch-9, step-2450 ---
loss: 0.0859



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 299.16it/s]

--- Eval epoch-9, step-2450 ---
pr_auc: 0.0725
roc_auc: 0.5369
f1: 0.0404
loss: 0.4394
Loaded best model



Evaluation: 100%|██████████| 30/30 [00:00<00:00, 291.37it/s]

{'pr_auc': 0.08721309061196421, 'roc_auc': 0.5811944141051645, 'f1': 0.0, 'loss': 0.2529959950596094}


In [11]:
model_no_pre = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    embedding_dim=256,
)

In [12]:
# STEP 4: define trainer
trainer = Trainer(model=model_no_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="roc_auc",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(4031, 256, padding_idx=0)
    (procedures): Embedding(1276, 256, padding_idx=0)
    (drugs): Embedding(472, 256, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
    (drugs): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(256, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=384, out_features=1, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7ff8f40ef130>
Monitor: roc_auc
Monitor criterion: max
Epochs: 10

Epoch 0 / 10: 100%|██████████| 245/245 [00:02<00:00, 108.49it/s]

--- Train epoch-0, step-245 ---
loss: 0.2682



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 320.92it/s]

--- Eval epoch-0, step-245 ---
pr_auc: 0.0857
roc_auc: 0.5966
f1: 0.0000
loss: 0.2269
New best roc_auc score (0.5966) at epoch-0, step-245



...

--- Train epoch-9, step-2450 ---
loss: 0.1380



Evaluation: 100%|██████████| 31/31 [00:00<00:00, 320.66it/s]

--- Eval epoch-9, step-2450 ---
pr_auc: 0.0906
roc_auc: 0.6324
f1: 0.0267
loss: 0.2902
Loaded best model



Evaluation: 100%|██████████| 30/30 [00:00<00:00, 314.23it/s]

{'pr_auc': 0.09722132504096945, 'roc_auc': 0.5907902948978, 'f1': 0.0, 'loss': 0.23656417181094488}
